# Quad Music Transformer Training Dataset Maker (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2024

***

# (SETUP ENVIRONMENT)

In [ ]:
#@title Install all dependencies (run only once per session)

!git clone https://github.com/asigalov61/tegridy-tools

In [ ]:
#@title Import all needed modules

print('Loading needed modules. Please wait...')
import os
import copy
import math
import statistics
import random

from joblib import Parallel, delayed, parallel_config

from tqdm import tqdm

if not os.path.exists('/content/Dataset'):
    os.makedirs('/content/Dataset')

print('Loading TMIDIX module...')
os.chdir('/content/tegridy-tools/tegridy-tools')

import TMIDIX

print('Done!')

os.chdir('/content/')
print('Enjoy! :)')

# (DOWNLOAD SOURCE MIDI DATASET)

In [ ]:
#@title Download Monster MIDI Subset

%cd /content/Dataset/

!wget 'https://huggingface.co/datasets/projectlosangeles/Monster-MIDI-Dataset/resolve/main/Monster_MIDI_Dataset_Search_Results_Ver_1_0_CC_BY_NC_SA.zip'
!unzip Monster_MIDI_Dataset_Search_Results_Ver_1_0_CC_BY_NC_SA.zip
!rm Monster_MIDI_Dataset_Search_Results_Ver_1_0_CC_BY_NC_SA.zip

%cd /content/

In [ ]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# (FILE LIST)

In [ ]:
#@title Save file list
###########

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')

dataset_addr = "/content/Dataset"
# os.chdir(dataset_addr)
filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

if filez == []:
    print('Could not find any MIDI files. Please check Dataset dir...')
    print('=' * 70)

print('Randomizing file list...')
random.shuffle(filez)

TMIDIX.Tegridy_Any_Pickle_File_Writer(filez, '/content/drive/MyDrive/filez')

In [ ]:
#@title Load file list
filez = TMIDIX.Tegridy_Any_Pickle_File_Reader('/content/drive/MyDrive/filez')
print('Done!')

# (LOAD TMIDIX MIDI PROCESSOR)

In [ ]:
# @title TMIDIX MIDI Processor

def TMIDIX_MIDI_Processor(midi_file):

    try:

        fn = os.path.basename(midi_file)

        #=======================================================
        # START PROCESSING

        #===============================================================================
        # Raw single-track ms score

        raw_score = TMIDIX.midi2single_track_ms_score(midi_file)

        #===============================================================================
        # Enhanced score notes

        escore_notes = TMIDIX.advanced_score_processor(raw_score, return_enhanced_score_notes=True)[0]

        if len(escore_notes) > 0:

            #=======================================================
            # PRE-PROCESSING

            # checking number of instruments in a composition
            instruments_list = list(set([y[3] for y in escore_notes]))

            #===============================================================================
            # Augmented enhanced score notes

            escore_notes = TMIDIX.augment_enhanced_score_notes(escore_notes)

            #=======================================================
            # Augmentation

            all_melody_chords = []

            for ta in range(0, 4, 2):
                for pa in range(-1, 2):

                    #=======================================================
                    # FINAL PROCESSING

                    melody_chords = []

                    # Break between compositions / Intro seq

                    if 9 in instruments_list:
                        drums_present = 2450 # Yes
                    else:
                        drums_present = 2449 # No

                    if escore_notes[0][3] != 9:
                        fpat = escore_notes[0][6]
                        fptc = escore_notes[0][4]+pa
                    else:
                        fpat = 128
                        fptc = escore_notes[0][4]

                    fchan = fpat // 8

                    melody_chords.extend([2595, drums_present, 2450+fchan, 2467+fptc]) # Intro seq + zero time

                    #=======================================================
                    # MAIN PROCESSING CYCLE
                    #=======================================================

                    pe = escore_notes[0]

                    chords_counter = 1

                    comp_chords_len = len(list(set([y[1] for y in escore_notes])))

                    for e in escore_notes:

                        #=======================================================
                        # Timings...

                        if (e[1] - pe[1]) != 0:
                            time = (e[1]-pe[1])+ta
                        else:
                            time = 0

                        dur = e[2]+ta

                        if time != 0 and time % 2 != 0:
                            time += 1
                        if dur % 2 != 0:
                            dur += 1

                        delta_time = int(max(0, min(255, time)) / 2)

                        # Durations and channels

                        dur = int(max(0, min(255, dur)) / 2)

                        cha = max(0, min(15, e[3]))

                        # Patches
                        if cha == 9: # Drums patch will be == 128
                            pat = 128

                        else:
                            pat = e[6]

                        # Channels

                        chan = pat // 8

                        # Pitches

                        if cha != 9:
                            ptc = max(1, min(127, e[4]+pa))
                        else:
                            ptc = max(1, min(127, e[4]))

                        # Velocities

                        # Calculating octo-velocity
                        velocity = max(8, min(127, e[5]))
                        vel = round(velocity / 8)-1

                        #=======================================================
                        # Outro seq

                        if comp_chords_len >= 250:
                            if ((comp_chords_len - chords_counter) == 50) and (delta_time != 0):
                                melody_chords.extend([2448, 2448, 2448, 2448]) # outro seq

                        if delta_time != 0:
                            chords_counter += 1

                        #=======================================================
                        # FINAL NOTE SEQ

                        # Writing final note asynchronously

                        cha_ptc = (128 * chan) + ptc

                        melody_chords.extend([delta_time, dur+128, cha_ptc+256, vel+2432])

                        pe = e

                        if len(melody_chords) > 8192:
                            break

                        #=======================================================

                    melody_chords.extend([2596]) # EOS

                    #=======================================================

                    # TOTAL DICTIONARY SIZE 2596+1=2597
                    #=======================================================

                    all_melody_chords.append(melody_chords[:8193])

            return all_melody_chords

        else:
            return None

    except Exception as e:
        print('=' * 70)
        print(midi_file)
        print(e)
        print('=' * 70)
        return None

# (PROCESS)

In [ ]:
#@title Process MIDIs with TMIDIX MIDI processor

NUMBER_OF_PARALLEL_JOBS = 16 # Number of parallel jobs
NUMBER_OF_FILES_PER_ITERATION = 16 # Number of files to queue for each parallel iteration
SAVE_EVERY_NUMBER_OF_ITERATIONS = 160 # Save every 2560 files

print('=' * 70)
print('TMIDIX MIDI Processor')
print('=' * 70)
print('Starting up...')
print('=' * 70)

###########

melody_chords_f = []

files_count = 0

print('Processing MIDI files. Please wait...')
print('=' * 70)

for i in tqdm(range(0, len(filez), NUMBER_OF_FILES_PER_ITERATION)):

  with parallel_config(backend='threading', n_jobs=NUMBER_OF_PARALLEL_JOBS, verbose = 0):

    output = Parallel(n_jobs=NUMBER_OF_PARALLEL_JOBS, verbose=0)(delayed(TMIDIX_MIDI_Processor)(f) for f in filez[i:i+NUMBER_OF_FILES_PER_ITERATION])

    for o in output:

        if o is not None:
            melody_chords_f.append(o)

    files_count += len(melody_chords_f)

    # Saving every 2560 processed files
    if i % (NUMBER_OF_FILES_PER_ITERATION * SAVE_EVERY_NUMBER_OF_ITERATIONS) == 0 and i != 0:
        print('SAVING !!!')
        print('=' * 70)
        print('Saving processed files...')
        print('=' * 70)
        print('Data check:', min(melody_chords_f[0][0]), '===', max(melody_chords_f[0][0]), '===', len(list(set(melody_chords_f[0][0]))), '===', len(melody_chords_f[0][0]))
        print('=' * 70)
        print('Processed so far:', files_count, 'out of', len(filez), '===', files_count / len(filez), 'good files ratio')
        print('=' * 70)
        count = str(files_count)
        TMIDIX.Tegridy_Any_Pickle_File_Writer(melody_chords_f, '/content/drive/MyDrive/QUAD_INTs_'+count)
        melody_chords_f = []

        print('=' * 70)

print('SAVING !!!')
print('=' * 70)
print('Saving processed files...')
print('=' * 70)
print('Data check:', min(melody_chords_f[0][0]), '===', max(melody_chords_f[0][0]), '===', len(list(set(melody_chords_f[0][0]))), '===', len(melody_chords_f[0][0]))
print('=' * 70)
print('Processed so far:', files_count, 'out of', len(filez), '===', files_count / len(filez), 'good files ratio')
print('=' * 70)
count = str(files_count)
TMIDIX.Tegridy_Any_Pickle_File_Writer(melody_chords_f, '/content/drive/MyDrive/QUAD_INTs_'+count)

print('=' * 70)

# (TEST INTS)

In [ ]:
#@title Test INTs

train_data1 = random.choice(melody_chords_f)[0]

print('Sample INTs', train_data1[:15])

out = train_data1

if len(out) != 0:

    song = out
    song_f = []

    time = 0
    dur = 0
    vel = 90
    pitch = 0
    channel = 0

    output_MIDI_patches = "0, 10, 19, 24, 35, 40, 52, 56, 65, 73, 87, 89, 98, 105, 117, 123" # @param {type:"string"}

    patches = [int(p) if 0 <= int(p) < 128 else 0 for p in output_MIDI_patches.split(',')]

    for ss in song:

        if 0 <= ss < 128:

            time += ss

        if 128 <= ss < 256:

            dur = (ss-128)

        if 256 <= ss < 2432:

            chan = (ss-256) // 128

            if chan < 9:
                channel = chan
            elif 9 < chan < 15:
                channel = chan+1
            elif chan == 15:
                channel = 15
            elif chan == 16:
                channel = 9

            pitch = (ss-256) % 128

        if 2432 <= ss < 2448:

            vel = (((ss-2432)+1) * 8)-1

            song_f.append(['note', time, dur, channel, pitch, vel ])

detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Quad Music Transformer',
                                                        output_file_name = '/content/Quad-Music-Transformer-Composition',
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=patches,
                                                        timings_multiplier=32
                                                        )

print('Done!')

# Congrats! You did it! :)